In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

import torch.nn as nn

from src.consts import *
from src.main import main, setup_torch, get_corpus
from src.model import RNNModel
from src.training import train, evaluate

from notebooks.utils import summary, check_cuda_mem

In [3]:
use_data_paralellization = False

In [4]:
setup_torch()
device = torch.device("cuda" if USE_CUDA else "cpu")
corpus = get_corpus()

# TODO remove these two lines
assert len(corpus.dictionary) == 602755
assert corpus.valid.size()[0] == 11606861

ntokens = len(corpus.dictionary)
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
                 TIED)
if use_data_paralellization or USE_DATA_PARALLELIZATION:
    model = CustomDataParallel(model)
else:
    model.to(device)
criterion = nn.CrossEntropyLoss()

In [4]:
train(model, corpus, criterion, device)

INFO 2019-05-22 18:36:03,663: | epoch   1 |   200/23759 batches | lr 20.00 | ms/batch 167.81 | loss  9.69 | ppl 16162.76
INFO 2019-05-22 18:36:11,870: -----------------------------------------------------------------------------------------
INFO 2019-05-22 18:36:11,870: Exiting from training early


FileNotFoundError: [Errno 2] No such file or directory: 'models/trained_models/model-2019-05-22 18:35:30.100518.pt'

In [ ]:
# timestamp = datetime.datetime.now()
# with open(MODEL_FILE_NAME.format(timestamp), 'wb') as f:
#     torch.save(model, f)

In [5]:
# with open(MODEL_FILE_NAME.format(timestamp), 'rb') as f:
with open('models/trained_models/model-2019-05-22 18:36:17.481233.pt', 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    model.rnn.flatten_parameters()

In [6]:
val_loss = evaluate(model, corpus, criterion, device)

> /media/gabrielamelo/Novo volume/Projects/portuguese_wsc/src/training.py(62)evaluate()
-> if not use_test_data:


(Pdb)  n


> /media/gabrielamelo/Novo volume/Projects/portuguese_wsc/src/training.py(64)evaluate()
-> full_data = batchify(corpus.valid, BATCH_SIZE, device)


(Pdb)  n


> /media/gabrielamelo/Novo volume/Projects/portuguese_wsc/src/training.py(68)evaluate()
-> with torch.no_grad():


...

> /media/gabrielamelo/Novo volume/Projects/portuguese_wsc/src/training.py(72)evaluate()
-> output, hidden = model(data.permute(1, 0), (hidden[0].permute(1, 0, 2).contiguous(),


(Pdb)  data[0]


tensor([  1516, 469834,  34258, 189892, 150927,   6438,  14651,  88816,   2488,
           697,    195,      6,      8,      0,   1241,  54975,      8,   9437,
        148727,      8], device='cuda:0')


(Pdb)  c


RuntimeError: CUDA error: device-side assert triggered

In [17]:
# /pytorch/aten/src/THC/THCTensorIndex.cu:362: void indexSelectLargeIndex(TensorInfo<T, IndexType>, 
# TensorInfo<T, IndexType>, TensorInfo<long, IndexType>, int, int, IndexType, IndexType, long) 
# [with T = float, IndexType = unsigned int, DstDim = 2, SrcDim = 2, IdxDim = -2, IndexIsMajor = true]: 
# block: [90,0,0], thread: [80,0,0] Assertion `srcIndex < srcSelectDimSize` failed.